In [2]:
import sys  
import os
import pandas as pd
import io
import imblearn
sys.path.insert(0, '../../../scripts')
sys.path.insert(0, '../../../dataset/')
import ngrams_scikitlearn
import hate_column_converter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import svm
from imblearn.over_sampling import RandomOverSampler

In [3]:
dataframe = pd.read_csv('../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
instagram_comments = new_dataframe['instagram_comments']
y = new_dataframe['hate_speech']

/workspaces/hate-speech-detection-python/train/notebooks-my-paper/svm-with-oversampling/../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-my-paper/svm-with-oversampling/../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_

In [4]:
X = ngrams_scikitlearn.get_vector(instagram_comments)

oversample = RandomOverSampler(random_state=0)
x_over, y_over = oversample.fit_resample(X, y)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_over, y_over, train_size=0.8, test_size=0.1, random_state=0)

In [6]:
clf = svm.SVC()
model = clf.fit(x_train.toarray(), y_train)
y_pred = model.predict(x_test.toarray())

print('F1 score 1:', f1_score(y_test, y_pred, labels=[0,1], pos_label=1))
print('F1 score 0:', f1_score(y_test, y_pred, labels=[0,1], pos_label=0))
print('F1 score media 0 e 1:', f1_score(y_test, y_pred, labels=[0,1], average='macro'))

F1 score 1: 0.9798449612403101
F1 score 0: 0.9788617886178863
F1 score media 0 e 1: 0.9793533749290981


In [ ]:
import joblib

model_filename = '../../../../models/generated-models-base-paper/offensive_svm_ngrams_model.pkl'
joblib.dump(model, model_filename)